### reproduce the Flaxman et al analysis

https://github.com/ImperialCollegeLondon/covid19model/releases

Estimating the number of infections and the impact of non-pharmaceutical interventions on COVID-19 in 11 European countries

https://www.imperial.ac.uk/media/imperial-college/medicine/sph/ide/gida-fellowships/Imperial-College-COVID19-Europe-estimates-and-NPI-impact-30-03-2020.pdf

Try to reproduce the `base.r` code -- sets up the stan model, 

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime

from statsmodels.distributions.empirical_distribution import ECDF


In [2]:
from scipy.stats import gamma

def poly(x, p):
    """ 
    equivalent of R `poly function. See 
    https://stackoverflow.com/questions/41317127/python-equivalent-to-r-poly-function
    """
    x = np.array(x)
    X = np.transpose(np.vstack(list(x**k for k in range(p+1))))
    return np.linalg.qr(X)[0][:,1:]

def decimal_date(date):
    return date.year + (date.dayofyear -1 )/(365+date.is_leap_year)



def pgammaAlt(q, mean, cv=1):
    """
    alternative parameterization of gamma cdf
    see http://search.r-project.org/library/EnvStats/html/GammaAlt.html
    """
    shape = 1/cv**2
    scale = mean/shape
    return gamma.cdf(q, shape, scale=scale)

def rgammaAlt(n, mean, cv=1):
    """
    alternative parameterization of gamma random numbers
    see http://search.r-project.org/library/EnvStats/html/GammaAlt.html
    """
    shape = 1/cv**2
    scale = mean/shape
    return gamma.rvs(shape, scale=scale, size=n)



In [3]:
countries = [  "Denmark",
  "Italy",
  "Germany",
  "Spain",
  "United_Kingdom",
  "France",
  "Norway",
  "Belgium",
  "Austria", 
  "Sweden",
  "Switzerland"]

#countries = ['United_Kingdom',]

ddir = "../covid19model/data"

In [4]:
d = pd.read_csv('https://opendata.ecdc.europa.eu/covid19/casedistribution/csv', parse_dates=['dateRep'], dayfirst=True)

d = pd.read_csv(f"{ddir}/COVID-19-up-to-date.csv", parse_dates=['dateRep'], dayfirst=True, encoding='ISO-8859-1')

In [5]:
## convert to decimal date, although not actually needed?
d['t'] = decimal_date(d.dateRep.dt)

d=d.rename(columns={
    "countriesAndTerritories": "Countries_and_territories", 
    "deaths": "Deaths", 
    "cases": "Cases",
    "dateRep": "DateRep"});

In [6]:
ifr_by_country = pd.read_csv(f"{ddir}/weighted_fatality.csv", )
ifr_by_country['country'] = ifr_by_country.iloc[:,1]
ifr_by_country.loc[ifr_by_country['country']=='United Kingdom', 'country'] = 'United_Kingdom'

In [7]:
serial_interval = pd.read_csv(f"{ddir}/serial_interval.csv")
covariates = pd.read_csv(f"{ddir}/interventions.csv", parse_dates=[1,2,3,4,5,6,7], dayfirst=True, nrows=11)
covariates = covariates.iloc[:11, :8]

In [8]:
## need to check this -- gives multiple SettingWithCopyWarning w/o *.loc
covariates.schools_universities.loc[covariates.schools_universities > covariates.lockdown] = covariates.lockdown.loc[covariates.schools_universities > covariates.lockdown]
covariates.travel_restrictions.loc[covariates.travel_restrictions > covariates.lockdown] = covariates.lockdown.loc[covariates.travel_restrictions > covariates.lockdown] 
covariates.public_events.loc[covariates.public_events > covariates.lockdown] = covariates.lockdown.loc[covariates.public_events > covariates.lockdown]
covariates.sport.loc[covariates.sport > covariates.lockdown] = covariates.lockdown.loc[covariates.sport > covariates.lockdown]
covariates.social_distancing_encouraged.loc[covariates.social_distancing_encouraged > covariates.lockdown] = covariates.lockdown.loc[covariates.social_distancing_encouraged > covariates.lockdown]
covariates.self_isolating_if_ill.loc[covariates.self_isolating_if_ill > covariates.lockdown] = covariates.lockdown.loc[covariates.self_isolating_if_ill > covariates.lockdown]


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
p = covariates.shape[1]   # number of columns
forecast = 0

In [10]:
DEBUG = False
if not DEBUG:
  N2 = 75 # Increase this for a further forecast
else:
  ### For faster runs:
  # countries = c("Austria","Belgium") #,Spain")
  N2 = 75

# countries = c("Italy","United_Kingdom","Spain","Norway","Austria","Switzerland")

dates = {}
reported_cases = {}
stan_data = {'M': len(countries), 
             'N': [],
             'p': p,
             'x1': poly(range(N2),2)[:,0],
             'x2': poly(range(N2),2)[:,1],
             'y': [],
             'covariate1': [],
             'covariate2': [],
             'covariate3': [],
             'covariate4': [],
             'covariate5': [],
             'covariate6': [],
             'covariate7': [],
             'deaths': [],
             'f': [],
             'N0': 6,
             'cases': [],
             'LENGTHSCALE': 7,
             'SI': serial_interval.fit[:N2],
             'EpidemicStart': []} # N0 = 6 to make it consistent with Rayleigh
deaths_by_country = {}


In [11]:
for Country in countries:

    IFR=ifr_by_country.weighted_fatality[ifr_by_country.country == Country].values[0]

    covariates1 = covariates[covariates.Country == Country].iloc[:,1:]

    d1 = d[d.Countries_and_territories==Country]
    d1 = d1.sort_values('t')
    index = np.where(d1.Cases>0)[0][0]
    index1 = np.where(np.cumsum(d1.Deaths)>=10)[0][0] # also 5
    index2 = index1-30
    print(f"First non-zero cases is on day {index}, and 30 days before 5 days is day {index2}")
    d1=d1.iloc[index2:]
    stan_data['EpidemicStart'].append(index1+1-index2) 

    for cov in covariates1.columns:
            d1[cov] =  (d1.DateRep >= covariates1.loc[:,cov].values[0])  # should this be > or >=?
            ### AHJ: need to coerce to just a value not a series for the comparison
            
            
    dates[Country] = d1.DateRep
    # hazard estimation
    N = len(d1.Cases)
    print(f"{Country} has {N} days of data")
    forecast = N2 - N
    if forecast < 0:
        print(f"{Country}: {N}")
        print("ERROR!!!! increasing N2")
        N2 = N
        forecast = N2 - N


    print("N, N2, forecast = ", N, N2, forecast)
    h = []
    DEBUG = False
    if DEBUG:  # OLD -- but faster for testing this part of the code
        mean = 18.8
        cv = 0.45

        for i in range(1, forecast+N+1):
            h1 = (IFR*pgammaAlt(i, mean, cv=cv) - IFR*pgammaAlt(i-1, mean, cv=cv))/(1-IFR*pgammaAlt(i-1, mean, cv=cv))
            h.append(h1)
    else:
        mean1 = 5.1; cv1 = 0.86; # infection to onset
        mean2 = 18.8; cv2 = 0.45 # onset to death
        ## assume that IFR is probability of dying given infection
        x1 = rgammaAlt(int(5e6), mean1, cv1) # infection-to-onset ----> do all people who are infected get to onset?
        x2 = rgammaAlt(int(5e6), mean2, cv2) # onset-to-death
        fc = ECDF(x1+x2)
        convolution = lambda u: IFR * fc(u)

        h.append(convolution(1.5) - convolution(0)) 
        for i in range(2, forecast+N+1):
            h.append((convolution(i+.5) - convolution(i-.5)) / (1-convolution(i-.5)))
        h = np.array(h)
            
    s = [1,] 
    for i in range(1,N2):
        s.append(s[i-1]*(1-h[i-1]))
    s = np.array(s)

    f = s * h

    y = np.hstack((d1.Cases.values,-np.ones(forecast)))
    reported_cases[Country] = d1.Cases
    deaths = np.hstack((d1.Deaths.values, -np.ones(forecast)))
    cases = np.hstack((d1.Cases.values, -np.ones(forecast)))
    deaths_by_country[Country] = d1.Deaths
    covariates2 = d1[covariates1.columns]
    # x=1:(N+forecast)    
    ### append copies of the last row so that it has length N+forecast
    covariates2 = covariates2.append(pd.DataFrame(covariates2.iloc[-1:].values.repeat(forecast, axis=0), 
                                                  columns=covariates2.columns, index=np.arange(N, N+forecast)))
    #     ## append data
    stan_data['N'].append(N)
    stan_data['y'].append(y[1]) # just the index case!
    # stan_data.x = cbind(stan_data.x,x)
    stan_data['covariate1'].append(covariates2.iloc[:,0].values)
    stan_data['covariate2'].append(covariates2.iloc[:,1].values)
    stan_data['covariate3'].append(covariates2.iloc[:,2].values)
    stan_data['covariate4'].append(covariates2.iloc[:,3].values) 
    stan_data['covariate5'].append(covariates2.iloc[:,4].values)
    stan_data['covariate6'].append(covariates2.iloc[:,5].values)
    stan_data['covariate7'].append(covariates2.iloc[:,6].values)
    stan_data['f'].append(f)
    stan_data['deaths'].append(deaths)
    stan_data['cases'].append(cases)

    stan_data['N2'] = N2
    stan_data['x'] = np.arange(N2)
    

stan_data['covariate2'] *= 0 # remove travel bans
stan_data['covariate4'] *= 0 # remove sport

#stan_data$covariate1 = stan_data$covariate1 # school closure
stan_data['covariate2'] = stan_data['covariate7'] # self-isolating if ill
#stan_data$covariate3 = stan_data$covariate3 # public events
# create the `any intervention` covariate
stan_data['covariate4'] = (stan_data['covariate1']+
                           stan_data['covariate3']+
                           stan_data['covariate5']+
                           stan_data['covariate6']+
                           stan_data['covariate7'])
# stan_data$covariate5 = stan_data$covariate5 # lockdown
# stan_data$covariate6 = stan_data$covariate6 # social distancing encouraged
stan_data['covariate7'] = 0 # models should only take 6 covariates

First non-zero cases is on day 58, and 30 days before 5 days is day 52
Denmark has 37 days of data
N, N2, forecast =  37 75 38
First non-zero cases is on day 31, and 30 days before 5 days is day 27
Italy has 62 days of data
N, N2, forecast =  62 75 13
First non-zero cases is on day 28, and 30 days before 5 days is day 46
Germany has 43 days of data
N, N2, forecast =  43 75 32
First non-zero cases is on day 32, and 30 days before 5 days is day 40
Spain has 49 days of data
N, N2, forecast =  49 75 26
First non-zero cases is on day 31, and 30 days before 5 days is day 43
United_Kingdom has 46 days of data
N, N2, forecast =  46 75 29
First non-zero cases is on day 25, and 30 days before 5 days is day 38
France has 51 days of data
N, N2, forecast =  51 75 24
First non-zero cases is on day 58, and 30 days before 5 days is day 55
Norway has 34 days of data
N, N2, forecast =  34 75 41
First non-zero cases is on day 35, and 30 days before 5 days is day 49
Belgium has 40 days of data
N, N2, fore

In [12]:
dbg = True
if dbg:
    
    for i, co in enumerate(countries):
        nr = slice(0,stan_data['N'][i])
        with open(f"results/{co}-check-dates-python.csv", 'w') as f:
            pd.DataFrame({
                'date': dates[co],
                'school closure': stan_data['covariate1'][i][nr],
                'self isolating if ill': stan_data['covariate2'][i][nr],
                'public event': stan_data['covariate3'][i][nr],
                'government makes any intervention': stan_data['covariate4'][i][nr],
                'lockdown': stan_data['covariate5'][i][nr],
                'social distancing encouraged': stan_data['covariate6'][i][nr]
              }).to_csv(f)

# stan_data['y'] = stan_data['y'].transpose()
# m = stan_model(f'stan-models/{StanModel}.stan')

# if(DEBUG) {
#   fit = sampling(m,data=stan_data,iter=40,warmup=20,chains=2)
# } else { 
#   # fit = sampling(m,data=stan_data,iter=4000,warmup=2000,chains=8,thin=4,control = list(adapt_delta = 0.90, max_treedepth = 10))
#   fit = sampling(m,data=stan_data,iter=200,warmup=100,chains=4,thin=4,control = list(adapt_delta = 0.90, max_treedepth = 10))
# }  